# CMSC 461 Project Notebook

**Name:** Robert Rose  
**Semester:** Spring 2019

User Guide Note: To run a cell, hit shift and enter.

First ensure that the connector is installed via pip:

In [1]:
!pip install mysql-connector-python

You are using pip version 19.0.2, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import mysql.connector
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Get database password:

In [3]:
import getpass
db_user = 'root'
db_password = getpass.getpass()

········


In [5]:
config = {
    'user': db_user,
    'password': db_password,
    'host': 'localhost',
    'port': 3306,
    'database': 'project',
    'raise_on_warnings': False
}

In [7]:
db_connection = mysql.connector.connect(**config)

In [8]:
cur = db_connection.cursor()

## Reports

### 1. List the unique common names of all the vegetables currently planted

In [34]:
def report_01():
    query = """
    SELECT DISTINCT plants.common_name 
    FROM pots JOIN plants ON (pots.holding_species, pots.holding_cultivar) = (plants.species, plants.cultivar) 
    WHERE plants.plant_type = 'vegetables';
    """
    try:
        cur.execute(query)
        names = cur.fetchall()
        for name in names:
            print(name[0])
    except Exception as err:
        print(f"An error {err} occurred")
report_01()

Bok choy
Turnip
Zucchini
Pumpkin


### 2. List the species name and number of varieties for each plant species for all the plants.

In [35]:
def report_02():
    query = """
    SELECT species, COUNT(cultivar) AS num_varieties FROM plants
    GROUP BY species;
    """
    try:
        cur.execute(query)
        results = cur.fetchall()
        for res in results:
            print(f"{res[0]}: {res[1]}")
    except Exception as err:
        print(f"An error {err} occurred")
report_02()

Allium sativum: 2
Beta vulgaris: 2
Brassica oleracea: 2
Brassica rapa: 2
Calendula officinalis: 2
Capsicum annuum: 2
Chrysanthemum morifolium: 2
Cucurbita pepo: 2
Helianthus annuus: 2
Hemerocallis lilioasphodelus: 2
Ismelia carinata: 2
Mentha × piperita: 2
Nepeta cataria: 2
Ocimum basilicum: 2
Origanum vulgare: 2
Rosmarinus officinalis: 2
Trientalis latifolia: 2
Tulipa × gesneriana: 2


### 3. Find the total number of pots currently holding a plant with a given common name

In [36]:
@interact_manual(common_name='Case-Sensitive Common Name')
def report_03(common_name):
    query = """
    SELECT COUNT(pots.id) AS TOTAL
    FROM pots JOIN plants ON (pots.holding_species, pots.holding_cultivar) = (plants.species, plants.cultivar) 
    WHERE plants.common_name = %s;
    """
    try:
        cur.execute(query, (common_name,))
        results = cur.fetchone()
        print(results[0])
    except Exception as err:
        print(f"An error {err} occurred")

interactive(children=(Text(value='Insert Case-Sensitive Common Name Here', description='common_name'), Button(…

### 4. Find the histogram (value and count) of volumes of pots with germinated plants with a given common name.

In [37]:
@interact_manual(common_name='Case-Sensitive Common Name')
def report_04(common_name):
    query = """
    SELECT pots.volume AS volume, COUNT(pots.id) AS NUM
    FROM pots JOIN plants ON (pots.holding_species, pots.holding_cultivar) = (plants.species, plants.cultivar) 
    WHERE plants.common_name = %s
    GROUP BY pots.volume;
    """
    try:
        cur.execute(query, (common_name,))
        results = cur.fetchall()
        for res in results:
            print(f"{res[0]}: {res[1]}")
    except Exception as err:
        print(f"An error {err} occurred")

interactive(children=(Text(value='Case-Sensitive Common Name', description='common_name'), Button(description=…

### 5. Find the most populous species among the pots with germinated plants.

In [40]:
def report_05():
    query = """
    SELECT pots.holding_species, COUNT(pots.id) AS NUM
    FROM pots
    WHERE pots.holding_germination_date IS NOT NULL
    GROUP BY pots.holding_species
    ORDER BY NUM DESC
    LIMIT 1;
    """
    try:
        cur.execute(query)
        results = cur.fetchone()
        print(f"{results[0]}: {results[1]} plants")
    except Exception as err:
        print(f"An error {err} occurred")
report_05()

Ismelia carinata: 7 plants


## CRUD Operations

### Barcodes

Barcodes must be 13 characters long. There is no need for UPDATE commands on the barcode because there's only one value in the table.

In [30]:
@interact_manual(barcode='13 CHARACTERS')
def create_barcode(barcode):
    query = f"""
    INSERT INTO barcodes(barcode)
    VALUES(%s)
    """
    if len(barcode) == 13:
        try:
            cur.execute(query, (barcode,))
            print(f"Barcode {barcode} successfully inserted!")
            db_connection.commit()
        except Exception as err:
            print(f"An error {err} occurred")
            db_connection.rollback()
    else:
        print("Barcode must be 13 characters long.")

interactive(children=(Text(value='13 CHARACTERS', description='barcode'), Button(description='Run Interact', s…

In [29]:
@interact(barcode='')
def search_barcode(barcode):
    # This searches to see if the barcode is in the database and what ids it corresponds to.
    query = """
    SELECT * FROM barcode_lookup_view
    WHERE barcode LIKE CONCAT (%s, '%')
    """
    print("('Barcode', 'Species', 'Cultivar', 'tray_id', 'pots_id', 'station_id')")
    cur.execute(query, (barcode,))
    results = cur.fetchall()
    for res in results:
        print(res)

interactive(children=(Text(value='', description='barcode'), Output()), _dom_classes=('widget-interact',))

In [32]:
@interact_manual(barcode='13 CHARACTERS')
def delete_barcode(barcode):
    query = """
    DELETE FROM barcodes WHERE barcode = %s
    """
    if len(barcode) == 13:
        try:
            cur.execute(query, (barcode,))
            print(f"Barcode {barcode} successfully deleted!")
            db_connection.commit()
        except Exception as err:
            print(f"An error {err} occurred")
            db_connection.rollback()
    else:
        print("Barcode must be 13 characters long.")

interactive(children=(Text(value='13 CHARACTERS', description='barcode'), Button(description='Run Interact', s…